In [1]:
import torch
import tensorly as tl
from tensorly.decomposition import partial_tucker
from tltorch import FactorizedConv
from torchvision.models import resnet18

/home/usainzg/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tl.set_backend('pytorch')

In [17]:
model = resnet18(weights=None)

In [18]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [30]:
resnet = ResNet18()

In [63]:
import copy

def tensorize_basic_block(basic_block, factorization="tucker", rank=0.5, decompose_weights=False):
    conv1 = basic_block.conv1
    conv2 = basic_block.conv2
    new_conv1 = FactorizedConv.from_conv(conv1, factorization=factorization, decompose_weights=decompose_weights, fixed_rank_modes='spatial', rank=rank)
    new_conv2 = FactorizedConv.from_conv(conv2, factorization=factorization, decompose_weights=decompose_weights, fixed_rank_modes='spatial', rank=rank)
    basic_block.conv1 = new_conv1
    basic_block.conv2 = new_conv2
    return basic_block

def tensorize_resnet(model, factorization="tucker", rank=0.5, decompose_weights=False):
    tensorized_model = copy.deepcopy(model)
    for layer in range(1, 5):
        basic_block_1 = getattr(tensorized_model, f'layer{layer}')[0]
        basic_block_2 = getattr(tensorized_model, f'layer{layer}')[1]
        basic_block_1 = tensorize_basic_block(basic_block_1, factorization=factorization, rank=rank, decompose_weights=decompose_weights)
        basic_block_2 = tensorize_basic_block(basic_block_2, factorization=factorization, rank=rank, decompose_weights=decompose_weights)
        setattr(tensorized_model, f'layer{layer}', torch.nn.Sequential(basic_block_1, basic_block_2))
    return tensorized_model

resnet_tn = tensorize_resnet(resnet, rank=0.8, decompose_weights=False)

In [31]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [40]:
resnet_tn

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(39, 39, 3, 3), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(39, 39, 3, 3))
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(39, 39, 3, 3), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(39, 39, 3, 3))
      )
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): FactorizedConv(
        in_channels=64, out_chann

In [35]:
x = torch.randn(1, 3, 32, 32)
y = resnet(x)
y_tn = resnet_tn(x)
print(y.shape)
print(y_tn.shape)

torch.Size([1, 10])
torch.Size([1, 10])


In [41]:
import numpy as np

def count_params(net: torch.nn.Module) -> np.array:
    return sum(p.numel() for p in net.parameters() if p.requires_grad)

In [44]:
print(f'resnet: {count_params(resnet)}')
print(f'resnet_tn: {count_params(resnet_tn)}')

print(f'compression ratio: {count_params(resnet) / count_params(resnet_tn)}')

resnet: 11173962
resnet_tn: 5690166
compression ratio: 1.9637321652830515


In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

torch.set_float32_matmul_precision("high")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [55]:
resnet = resnet.to(device)

In [56]:
optimizer = optim.SGD(resnet.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

def train(epoch, net):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

def test(epoch, net):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

In [57]:
for epoch in range(5):
    train(epoch, resnet)
    test(epoch, resnet)
    scheduler.step()


Epoch: 0
0 391 Loss: 1.682 | Acc: 38.281% (49/128)
1 391 Loss: 1.700 | Acc: 36.328% (93/256)
2 391 Loss: 1.698 | Acc: 35.938% (138/384)
3 391 Loss: 1.681 | Acc: 37.891% (194/512)
4 391 Loss: 1.662 | Acc: 38.750% (248/640)
5 391 Loss: 1.669 | Acc: 38.542% (296/768)
6 391 Loss: 1.646 | Acc: 40.290% (361/896)
7 391 Loss: 1.638 | Acc: 39.941% (409/1024)
8 391 Loss: 1.624 | Acc: 40.972% (472/1152)
9 391 Loss: 1.618 | Acc: 41.328% (529/1280)
10 391 Loss: 1.621 | Acc: 41.406% (583/1408)
11 391 Loss: 1.616 | Acc: 41.797% (642/1536)
12 391 Loss: 1.618 | Acc: 41.406% (689/1664)
13 391 Loss: 1.628 | Acc: 41.016% (735/1792)
14 391 Loss: 1.629 | Acc: 40.885% (785/1920)
15 391 Loss: 1.628 | Acc: 40.869% (837/2048)
16 391 Loss: 1.632 | Acc: 40.349% (878/2176)
17 391 Loss: 1.630 | Acc: 40.365% (930/2304)
18 391 Loss: 1.631 | Acc: 40.378% (982/2432)
19 391 Loss: 1.635 | Acc: 40.117% (1027/2560)
20 391 Loss: 1.636 | Acc: 40.141% (1079/2688)
21 391 Loss: 1.635 | Acc: 40.163% (1131/2816)
22 391 Loss: 1.6

In [59]:
resnet_acc = best_acc

In [64]:
resnet_tn = resnet_tn.to(device)

optimizer = optim.SGD(resnet_tn.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

for epoch in range(5):
    train(epoch, resnet_tn)
    test(epoch, resnet_tn)
    scheduler.step()


Epoch: 0
0 391 Loss: 2.411 | Acc: 8.594% (11/128)
1 391 Loss: 2.380 | Acc: 9.766% (25/256)
2 391 Loss: 2.381 | Acc: 9.635% (37/384)
3 391 Loss: 2.374 | Acc: 8.984% (46/512)
4 391 Loss: 2.371 | Acc: 8.594% (55/640)
5 391 Loss: 2.373 | Acc: 7.943% (61/768)
6 391 Loss: 2.373 | Acc: 7.701% (69/896)
7 391 Loss: 2.372 | Acc: 8.105% (83/1024)
8 391 Loss: 2.367 | Acc: 8.941% (103/1152)
9 391 Loss: 2.363 | Acc: 9.453% (121/1280)
10 391 Loss: 2.355 | Acc: 9.943% (140/1408)
11 391 Loss: 2.355 | Acc: 9.961% (153/1536)
12 391 Loss: 2.355 | Acc: 10.337% (172/1664)
13 391 Loss: 2.354 | Acc: 10.547% (189/1792)
14 391 Loss: 2.353 | Acc: 10.521% (202/1920)
15 391 Loss: 2.350 | Acc: 10.742% (220/2048)
16 391 Loss: 2.347 | Acc: 11.029% (240/2176)
17 391 Loss: 2.345 | Acc: 11.111% (256/2304)
18 391 Loss: 2.340 | Acc: 11.184% (272/2432)
19 391 Loss: 2.337 | Acc: 11.445% (293/2560)
20 391 Loss: 2.337 | Acc: 11.310% (304/2688)
21 391 Loss: 2.335 | Acc: 11.435% (322/2816)
22 391 Loss: 2.334 | Acc: 11.583% (34

KeyboardInterrupt: 

In [62]:
best_acc

53.5